### Some scrap code for testing out permutation testing for decoding by time, cross decoding by time, any other sig testing

In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import utils.visualization_utils as visualization_utils

import utils.io_utils as io_utils

import utils.glm_utils as glm_utils
import utils.stats_utils as stats_utils
from matplotlib import pyplot as plt
import matplotlib
import utils.spike_utils as spike_utils
from constants.behavioral_constants import *
from constants.decoding_constants import *
from scripts.pseudo_decoding.belief_partitions.belief_partition_configs import *
import scripts.pseudo_decoding.belief_partitions.belief_partitions_io as belief_partitions_io
import itertools

import argparse
import copy
from tqdm import tqdm


### by time decoding first

In [12]:
args = argparse.Namespace(
    **BeliefPartitionConfigs()._asdict()
)
args.trial_event = "StimOnset"
args.subject = "both"
args.mode = "pref"
args.region_level = "structure_level2_cleaned"
args.regions = "basal_ganglia_BG"
args.beh_filters = {"Response": "Correct", "Choice": "Chose"}
args.sig_unit_level = "pref_99th_window_filter_drift"
res = belief_partitions_io.read_results(args, FEATURES)


In [7]:
def get_n_time_offset(args):
    if args.trial_event == "StimOnset":
        n_time = 20
        offset = 0.9
    else: 
        n_time = 33
        offset = 1.7
    return n_time, offset

def compute_p_for_decoding_by_time(res, args): 
    res["shuffle_type"] = res["mode"].map({"pref": "true", "pref_shuffle": "shuffle"})
    n_time, offset = get_n_time_offset(args)
    p_res = []
    for time_idx in tqdm(range(n_time)):
        time = round(time_idx / 10 - offset, 1)
        time_res = res[np.isclose(res.Time, time)]
        p = stats_utils.compute_p_per_group(time_res, "Accuracy", "shuffle_type")
        p_res.append({"Time": time, "TimeIdx": time_idx, "p": p})
    p_res = pd.DataFrame(p_res)

def compute_p_for_cross_decoding_by_time(cross_res, shuffles, args): 
    n_time, offset = get_n_time_offset(args)
    for (train_idx, test_idx) in tqdm(itertools.product(range(n_time), range(n_time))):
        train_time = round(train_idx / 10 - offset, 1)
        test_time = round(test_idx / 10 - offset, 1)

        time_res = cross_res[np.isclose(cross_res.TrainTime, train_time) & np.isclose(cross_res.TestTime, test_time)].copy()
        time_res["shuffle_type"] = "true"
        shuffle_time_res = shuffles[np.isclose(shuffles.Time, test_time)].copy()
        shuffle_time_res["shuffle_type"] = "shuffle"

        p = stats_utils.compute_p_per_group(pd.concat((time_res, shuffle_time_res)), "Accuracy", "shuffle_type")
        p_res.append({"TrainTime": train_time, "TestTime": test_time, "TrainIdx": train_idx, "TestIdx": test_idx, "p": p})
    p_res = pd.DataFrame(p_res)

In [8]:
ps = compute_p_for_decoding_by_time(res, args)

100%|██████████| 33/33 [00:23<00:00,  1.43it/s]


In [13]:
res = belief_partitions_io.read_results(args, FEATURES)
shuffles = res[res["mode"] == f"{args.mode}_shuffle"]
cross_res = belief_partitions_io.read_cross_time_results(args, FEATURES)

In [19]:
cross_res

,TrainTime,TestTime,RunIdx,Feat,Accuracy,TrainEvent,TestEvent
0,-0.9,-0.9,0,CIRCLE,0.7075,StimOnset,StimOnset
1,-0.9,-0.9,1,CIRCLE,0.7000,StimOnset,StimOnset
2,-0.9,-0.9,2,CIRCLE,0.6200,StimOnset,StimOnset
3,-0.9,-0.9,3,CIRCLE,0.4275,StimOnset,StimOnset
4,-0.9,-0.9,4,CIRCLE,0.7375,StimOnset,StimOnset
...,...,...,...,...,...,...,...
38395,1.0,1.0,3,SWIRL,0.3550,StimOnset,StimOnset
38396,1.0,1.0,4,SWIRL,0.3425,StimOnset,StimOnset
38397,1.0,1.0,5,SWIRL,0.5100,StimOnset,StimOnset
38398,1.0,1.0,6,SWIRL,0.6000,StimOnset,StimOnset


In [21]:
p_res = []
n_time = 20
offset = 0.9

for (train_idx, test_idx) in tqdm(itertools.product(range(n_time), range(n_time))):
    train_time = round(train_idx / 10 - offset, 1)
    test_time = round(test_idx / 10 - offset, 1)

    time_res = cross_res[np.isclose(cross_res.TrainTime, train_time) & np.isclose(cross_res.TestTime, test_time)].copy()
    time_res["shuffle_type"] = "true"
    shuffle_time_res = shuffles[np.isclose(shuffles.Time, test_time)].copy()
    shuffle_time_res["shuffle_type"] = "shuffle"

    p = stats_utils.compute_p_per_group(pd.concat((time_res, shuffle_time_res)), "Accuracy", "shuffle_type")
    p_res.append({"TrainTime": train_time, "TestTime": test_time, "TrainIdx": train_idx, "TestIdx": test_idx, "p": p})
p_res = pd.DataFrame(p_res)

0it [00:00, ?it/s]

100%|██████████| 1000/1000 [00:00<00:00, 1340.25it/s]
400it [05:00,  1.33it/s]


In [24]:
p_res

,TrainTime,TestTime,TrainIdx,TestIdx,p
0,-0.9,-0.9,0,0,0.313
1,-0.9,-0.8,0,1,0.213
2,-0.9,-0.7,0,2,0.012
3,-0.9,-0.6,0,3,0.002
4,-0.9,-0.5,0,4,0.036
...,...,...,...,...,...
395,1.0,0.6,19,15,0.000
396,1.0,0.7,19,16,0.000
397,1.0,0.8,19,17,0.003
398,1.0,0.9,19,18,0.000


In [25]:
means = cross_res.groupby(["TrainTime", "TestTime"]).Accuracy.mean().reset_index(name="Accuracy")
merged = pd.merge(means, p_res)
pivoted = means.pivot(index="TrainTime", columns="TestTime", values="Accuracy")
merged = 